# CS 124 Programming Assignment 4: Information Retrieval (Winter 2025)

In this assignment you will be improving upon a rather poorly-made information retrieval system. You will build an inverted index to quickly retrieve documents that match queries and then make it even better by using term-frequency inverse-document-frequency weighting and cosine similarity to compare queries to your data set. Your IR system will be evaluated for accuracy on the correct documents retrieved for different queries and the correctly computed tf-idf values and cosine similarities.

## Data

You will be using your IR system to find relevant documents among a collection of sixty books sourced from Project Gutenberg. The training data is located in the data/ directory under the subdirectory ProjectGutenberg/. Within this directory you will see yet another directory raw/. This contains the raw text files of the sixty short stories. The data/ directory also contains the files dev_queries.txt and dev_solutions.txt. We have provided these to you as a set of development queries and their expected answers to use as you begin implementing your IR system.

## Your Task

Improve upon the given IR system by implementing the following features:

<b>Inverted Positional Index:</b> Implement an inverted index - a mapping from words to the documents in which they occur, as well as the positions in the documents for which they occur.

<b>Boolean Retrieval:</b> Implement a Boolean retrieval system, in which you return the list of documents that contain all words in a query. (Yes, you only need to support conjunctions for this assignment.)

<b>Phrase Query Retrieval:</b> Implement a system that returns the list of documents in which the full phrase appears, (ie. the words of the query appear next to each other, in the specified order). Note that at the time of retrieval, you will not have access to the original documents anymore (the documents would be turned into bag of words), so you'll have to utilize your inverted positional index to complete this part.

<b>TF-IDF:</b> Compute and store the term-frequency inverse-document-frequency value for every word-document co-occurrence.

<b>Cosine Similarity:</b> Implement cosine similarity in order to improve upon the ranked retrieval system, which currently retrieves documents based on the Jaccard coefficient between the query and each document. The reference solution uses <b> log tf-idf weighting </b> from the lecture for computing cosine scores. That is, $\cos(\theta) = \frac{\sum_{t \in Q \cap D} w_{t,d} \cdot w_{t,q}}{\sqrt{\sum_{t} w_{t,d}^2} \cdot \sqrt{\sum_{t} w_{t,q}^2}}$.


## Evaluation

Your IR system will be evaluated on a small development set of queries as well as a held-out set of queries. The development queries are encoded in the file <b>dev_queries.txt</b> and are:

- chest of drawers
- machine learning is cool
- the cold breeze
- pacific coast
- pumpkin pies
- i completed fizzbuzz

We test your system based on the five parts mentioned above: the inverted index (used both to get word positions and to get postings), boolean retrieval, phrase query retrieval, computing the correct tf-idf values, and implementing cosine similarity using the tf-idf values. The provided development queries contain some common words, some uncommon words, and the occasional non-existent word. Some of the query phrases are found verbatim in the book dataset, and some are not. Your system should be able to correctly handle all of these cases.  

### Environment Check

Before we do anything else, let's quickly check that you're running the correct
version of Python and are in the right environment!

In [1]:
import os
assert os.environ['CONDA_DEFAULT_ENV'] == "cs124"

import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 8

If the above cell complains, it means that you're using the wrong environment
or Python version!

If so, please exit this notebook, kill the notebook server with CTRL-C, and
try running

$ conda activate cs124

then restarting your notebook server with

$ jupyter notebook

If that doesn't work, you should go back and follow the installation
instructions in PA0.

## Getting Started!

We will first start by importing some modules and setting up our IR system class.


In [2]:
import json
import os
import re
import sys
import math

from porter_stemmer import PorterStemmer

In [3]:
class IRSystem:
    def __init__(self):
        # For holding the data - initialized in read_data()
        self.titles = []
        self.docs = []
        self.vocab = []
        # For the text pre-processing.
        self.alphanum = re.compile('[^a-zA-Z0-9]')
        self.p = PorterStemmer()

    def get_uniq_words(self):
        uniq = set()
        for doc in self.docs:
            for word in doc:
                uniq.add(word)
        return uniq

    def __read_raw_data(self, dirname):
        print("Stemming Documents...")

        titles = []
        docs = []
        os.mkdir('%s/stemmed' % dirname)
        title_pattern = re.compile('(.*) \d+\.txt')

        # make sure we're only getting the files we actually want
        filenames = []
        for filename in os.listdir('%s/raw' % dirname):
            if filename.endswith(".txt") and not filename.startswith("."):
                filenames.append(filename)

        for i, filename in enumerate(filenames):
            title = title_pattern.search(filename).group(1)
            print("    Doc %d of %d: %s" % (i + 1, len(filenames), title))
            titles.append(title)
            contents = []
            f = open('%s/raw/%s' % (dirname, filename), 'r', encoding="utf-8")
            of = open('%s/stemmed/%s.txt' % (dirname, title), 'w',
                      encoding="utf-8")
            for line in f:
                # make sure everything is lower case
                line = line.lower()
                # split on whitespace
                line = [xx.strip() for xx in line.split()]
                # remove non alphanumeric characters
                line = [self.alphanum.sub('', xx) for xx in line]
                # remove any words that are now empty
                line = [xx for xx in line if xx != '']
                # stem words
                line = [self.p.stem(xx) for xx in line]
                # add to the document's conents
                contents.extend(line)
                if len(line) > 0:
                    of.write(" ".join(line))
                    of.write('\n')
            f.close()
            of.close()
            docs.append(contents)
        return titles, docs

    def __read_stemmed_data(self, dirname):
        print("Already stemmed!")
        titles = []
        docs = []

        # make sure we're only getting the files we actually want
        filenames = []
        for filename in os.listdir('%s/stemmed' % dirname):
            if filename.endswith(".txt") and not filename.startswith("."):
                filenames.append(filename)

        if len(filenames) != 60:
            msg = "There are not 60 documents in ../data/ProjectGutenberg/stemmed/\n"
            msg += "Remove ../data/ProjectGutenberg/stemmed/ directory and re-run."
            raise Exception(msg)

        for i, filename in enumerate(filenames):
            title = filename.split('.')[0]
            titles.append(title)
            contents = []
            f = open('%s/stemmed/%s' % (dirname, filename), 'r',
                     encoding="utf-8")
            for line in f:
                # split on whitespace
                line = [xx.strip() for xx in line.split()]
                # add to the document's conents
                contents.extend(line)
            f.close()
            docs.append(contents)

        return titles, docs

    def read_data(self, dirname):
        """
        Given the location of the 'data' directory, reads in the documents to
        be indexed.
        """
        # NOTE: We cache stemmed documents for speed
        #       (i.e. write to files in new 'stemmed/' dir).

        print("Reading in documents...")
        # dict mapping file names to list of "words" (tokens)
        filenames = os.listdir(dirname)
        subdirs = os.listdir(dirname)
        if 'stemmed' in subdirs:
            titles, docs = self.__read_stemmed_data(dirname)
        else:
            titles, docs = self.__read_raw_data(dirname)

        # Sort document alphabetically by title to ensure we have the proper
        # document indices when referring to them.
        ordering = [idx for idx, title in sorted(enumerate(titles),
                                                 key=lambda xx: xx[1])]

        self.titles = []
        self.docs = []
        numdocs = len(docs)
        for d in range(numdocs):
            self.titles.append(titles[ordering[d]])
            self.docs.append(docs[ordering[d]])

        # Get the vocabulary.
        self.vocab = [xx for xx in self.get_uniq_words()]

You will first build the inverted positional index (data structure that keeps track of the documents in which a particular word is contained, and the positions of that word in the document). The documents will have already been read in at this point. The following instance variables in the class are included in the starter code for you to use to build your inverted positional index: titles (a list of strings), docs (a list of lists of strings), and vocab (a list of strings). Since the majority of your work in this assignment will use document ID, we recommend doing the mapping using the document IDs (i.e. the index of the document within `self.docs`). You can then retrieve the text and title of any given document by indexing into `self.docs` and `self.titles` respectively.

In [13]:
class IRSystem(IRSystem):
    def index(self):
        """
        Build an index of the documents.
        """
        print("Indexing...")
        # ------------------------------------------------------------------
        # TODO: Create an inverted, positional index.
        #       Granted this may not be a linked list as in a proper
        #       implementation.
        #       This index should allow easy access to both 
        #       1) the documents in which a particular word is contained, and 
        #       2) for every document, the positions of that word in the document 
        #       Some helpful instance variables:
        #         * self.docs = List of documents
        #         * self.titles = List of titles

        inv_index = {}

        # Generate inverted index here
        for i in range(len(self.docs)):
            docwords = self.docs[i]
            for j in range(len(docwords)):
                word = docwords[j]
                if word not in inv_index:
                    inv_index[word] = [[] for _ in range(len(self.docs))]
                    inv_index[word][i].append(j)
                else:
                    inv_index[word][i].append(j)
                

        self.inv_index = inv_index

        # ------------------------------------------------------------------

        # turn self.docs into a map from ID to bag of words
        id_to_bag_of_words = {}
        for d, doc in enumerate(self.docs):
            bag_of_words = set(doc)
            id_to_bag_of_words[d] = bag_of_words
        self.docs = id_to_bag_of_words

Next we will implement `get_word_positions`. This method returns a list of integers that identifies the positions in the document `doc` (represented as document ID) in which the word is found.  This is basically just an API into your inverted index, but you must implement it in order for the index to be evaluated fully. 

**Be careful when accessing your inverted index!** Trying to index into a dictionary using a key that is not present will cause an error (and may cause your submission to crash the autograder).

In [29]:
class IRSystem(IRSystem):
    def get_word_positions(self, word, doc):
        """
        Given a word and a document, use the inverted index to return
        the positions of the specified word in the specified document.
        """
        # ------------------------------------------------------------------
        # TODO: return the list of positions for a word in a document.
        positions = []
        if word not in self.inv_index:
            return []
        positions = self.inv_index[word][doc]

        return positions
        # ------------------------------------------------------------------

We will add another method, `get_posting`, that returns a list of integers (document IDs) that identifies the documents in which the word is found. This is basically another API into your inverted index, but you must implement it in order to be evaluated fully.

Keep in mind that the document IDs in each postings list to be sorted in order to perform the linear merge for boolean retrieval.

In [32]:
class IRSystem(IRSystem):
    def get_posting(self, word):
        """
        Given a word, this returns the list of document indices (sorted) in
        which the word occurs.
        """
        # ------------------------------------------------------------------
        # TODO: return the list of postings for a word.
        posting = []
        if word not in self.inv_index:
            return []
        occurences = self.inv_index[word]
        for i in range(len(occurences)):
            if len(occurences[i]) > 0:
                posting.append(i)
        posting.sort()

        return posting
        # ------------------------------------------------------------------
    
    def get_posting_unstemmed(self, word):
        """
        Given a word, this *stems* the word and then calls get_posting on the
        stemmed word to get its postings list. You should *not* need to change
        this function. It is needed for submission.
        """
        word = self.p.stem(word)
        return self.get_posting(word)

Next, we will implement Boolean retrieval, returning a list of document IDs corresponding to the documents in which all the words in query occur.

You're welcome to implement the linear merge algorithm outlined in the videos/book or any other method you prefer, but do not use built-in set intersection functions.

In [34]:
class IRSystem(IRSystem):
    def boolean_retrieve(self, query):
        """
        Given a query in the form of a list of *stemmed* words, this returns
        the list of documents in which *all* of those words occur (ie an AND
        query).
        Return an empty list if the query does not return any documents.
        """
        # ------------------------------------------------------------------
        # TODO: Implement Boolean retrieval. You will want to use your
        #       inverted index that you created in index().
        # Right now this just returns all the possible documents!
        docs = []
        words = query
        for i in range(len(self.docs)):
            occurrences_of_each_word_in_query = []
            #check if this doc has all the words
            for word in words:
                occurrences = self.get_word_positions(word, i)
                occurrences_of_each_word_in_query.append(len(occurrences))
            
            #if number of occurences for each word is greater than 0, then can add
            all_exist = True
            for occ in occurrences_of_each_word_in_query:
                if occ <= 0:
                    all_exist = False
                    break
            
            if all_exist:
                docs.append(i)

        # ------------------------------------------------------------------

        return sorted(docs)  # sorted doesn't actually matter

Let's continue to phase query retrieval. Our `phrase_retrieve` method will return a list of document IDs corresponding to the documents in which all the actual query phrase occurs.

In [41]:
class IRSystem(IRSystem):
    def phrase_retrieve(self, query):
        """
        Given a query in the form of an ordered list of *stemmed* words, this 
        returns the list of documents in which *all* of those words occur, and 
        in the specified order. 
        Return an empty list if the query does not return any documents. 
        """
        # ------------------------------------------------------------------
        # TODO: Implement Phrase Query retrieval (ie. return the documents 
        #       that don't just contain the words, but contain them in the 
        #       correct order) You will want to use the inverted index 
        #       that you created in index(), and may also consider using
        #       boolean_retrieve. 
        #       NOTE that you no longer have access to the original documents
        #       in self.docs because it is now a map from doc IDs to set
        #       of unique words in the original document.
        # Right now this just returns all possible documents!
        docs = []
        querywords = query
        
        #all words must occur in the doc first
        docs_with_the_words = self.boolean_retrieve(query)
        
        def helpfunc(index_lists, words):
            for start_idx in index_lists[0]:
                if all(start_idx + i in index_lists[i] for i in range(1, len(words))):
                    return True
            return False
        
        for d in docs_with_the_words:
            #get the positions of the words 
            index_lists = [self.inv_index[word][d] for word in querywords]
            dochasit = helpfunc(index_lists, querywords)
            if dochasit:
                docs.append(d)
            

        # ------------------------------------------------------------------

        return sorted(docs)  # sorted doesn't actually matter

Now we will compute and score the tf-idf values. compute_tfidf and stores the tf-idf values for words and documents. For this you will probably want to be aware of the class variable vocab, which holds the list of all unique words, as well as the inverted index you created earlier.

You must also implement `get_tfidf` to return the tf-idf weight for a particular word and document ID. Make sure you correctly handle the case where the word isn't present your vocabulary! 

In [43]:
class IRSystem(IRSystem):
    def compute_tfidf(self):
        # -------------------------------------------------------------------
        # TODO: Compute and store TF-IDF values for words and documents in self.tfidf.
        #       Recall that you can make use of:
        #         * self.vocab: a list of all distinct (stemmed) words
        #         * self.docs: a dictionary mapping document ID to a 
        #                       the set of words that appear in the doc
        #         * self.get_word_positions(word, doc): returns list of 
        #                       positions (i.e. all occurrences) of the given
        #                       word within the given document 
        #         * self.get_posting(word): returns the list of document IDs
        #                       for docs containing the given word
        #       NOTE that you probably do *not* want to store a value for every
        #       word-document pair, but rather just for those pairs where a
        #       word actually occurs in the document.
        print("Calculating tf-idf...")
        self.tfidf = {}
        self.idf = {}
        
        tf_map = {}
        idf_map = {}
        n = -1
        for word in self.inv_index:
            docs = self.inv_index[word]
            n = len(docs)
            tfvals = []
            
            number_of_documents_in_which_term_occurs = 0
            for d in docs:
                if len(d) == 0:
                    tfvals.append(0)
                else:
                    tfvals.append(1 + math.log10(len(d)))
                    number_of_documents_in_which_term_occurs += 1
            tf_map[word] = tfvals
            idf_map[word] = math.log10(n / number_of_documents_in_which_term_occurs)
            
        tfidf_map = {}
        for word in tf_map:
            tfidflist = [x * idf_map[word] for x in tf_map[word]]
            tfidf_map[word] = tfidflist
            
        self.tfidf = tfidf_map
        self.idf = idf_map
        
        
        # ------------------------------------------------------------------

    def get_tfidf(self, word, document):
        # ------------------------------------------------------------------
        # TODO: Return the tf-idf weigthing for the given word (string) and
        #       document index.
        tfidf = 0.0
        if word not in self.tfidf:
            return 0
        tfidf = self.tfidf[word][document]
        # ------------------------------------------------------------------
        return tfidf

    def get_tfidf_unstemmed(self, word, document):
        """
        This function gets the TF-IDF of an *unstemmed* word in a document.
        Stems the word and then calls get_tfidf. You should *not* need to
        change this interface, but it is necessary for submission.
        """
        word = self.p.stem(word)
        return self.get_tfidf(word, document)

Lastly, we will implement `rank_retrieve`. This function returns sorted list of the top ranked documents for a given query. Right now it ranks documents according to their Jaccard similarity with the query, but you will replace this method of ranking with a ranking using the <b>cosine similarity</b> between the documents and query.
    
Remember to use log TF-IDF weighting for both the query and the document when computing cosine similarity. This means that the query term weights will be computed as: $w_{t,q} = (1 + \log_{10} \operatorname{count}(t, q)) \cdot \operatorname{IDF}(t)$, where IDF (inverse document frequency) is: $\operatorname{IDF}(t) = \log_{10} \left( \frac{N}{df_t} \right)$, where $N$ is the total number of documents and $df_t$ is the number of documents containing term $t$. The document term weights will be: $w_{t,d} = \operatorname{TF-IDF}(t, d)$, where TF-IDF for the document is: $\operatorname{TF-IDF}(t, d) = (1 + \log_{10} \operatorname{count}(t, d)) \cdot \operatorname{IDF}(t)$. Finally, cosine similarity is computed as the dot product of the document and query vectors, normalized by their magnitudes: $\cos(\theta) = \frac{\sum_{t \in Q \cap D} w_{t,d} \cdot w_{t,q}}{\sqrt{\sum_{t} w_{t,d}^2} \cdot \sqrt{\sum_{t} w_{t,q}^2}}$. Refer to the lecture for a more detailed explanation.

In [53]:
class IRSystem(IRSystem):
    def rank_retrieve(self, query):
        """
        Given a query (a list of words), return a rank-ordered list of
        documents (by ID) and score for the query.
        """
#         scores = [0.0 for xx in range(len(self.titles))]
        # ------------------------------------------------------------------
        # TODO: Implement cosine similarity between a document and a list of
        #       query words.

        # Right now, this code is a placeholder that simply gets the score by
        # taking the Jaccard similarity between the query and every document. 
        # Please replace with your implementation of cosine similarity-based retrieval.
        query_words = query
        querymap = {}
        for qw in query_words:
            if qw not in querymap:
                querymap[qw] = 1
            else:
                querymap[qw] += 1
                
        tfidf_query = {}
        q_denom = 0
        for qw in querymap:
            tf = 1 + math.log10(querymap[qw])
            if qw not in self.idf:
                idf = 0
            else:
                idf = self.idf[qw]
            tfidf_query[qw] = tf * idf
            q_denom += (tfidf_query[qw])**2
        q_denom = math.sqrt(q_denom)
            
        scores = {}
        for i in range(len(self.titles)):
            
            unique_words_in_doc = self.docs[i]
            d_denom = 0
            for w in unique_words_in_doc:
                d_denom += self.tfidf[w][i] ** 2
            d_denom = math.sqrt(d_denom)
            
            curscore = 0
            for qw in query_words:
                num1 = tfidf_query[qw]
                if qw not in self.tfidf:
                    num2 = 0
                else:
                    num2 = self.tfidf[qw][i]
                curscore += ((num1 / q_denom) * (num2 / d_denom))
            scores[i] = curscore
                
        sorted_keys = [k for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)]
        rankings = []
        for i in sorted_keys:
            rankings.append((i, scores[i]))
        return rankings
        
        
        
        
        

Let's also add a few methods that given a string, will process and then return the list of matching documents for the different methods you have implemented. You do not need to add any additional code here.

In [20]:
class IRSystem(IRSystem): 
    def process_query(self, query_str):
        """
        Given a query string, process it and return the list of lowercase,
        alphanumeric, stemmed words in the string.
        """
        # make sure everything is lower case
        query = query_str.lower()
        # split on whitespace
        query = query.split()
        # remove non alphanumeric characters
        query = [self.alphanum.sub('', xx) for xx in query]
        # stem words
        query = [self.p.stem(xx) for xx in query]
        return query

    def query_retrieve(self, query_str):
        """
        Given a string, process and then return the list of matching documents
        found by boolean_retrieve().
        """
        query = self.process_query(query_str)
        return self.boolean_retrieve(query)

    def phrase_query_retrieve(self, query_str):
        """
        Given a string, process and then return the list of matching documents
        found by phrase_retrieve().
        """
        query = self.process_query(query_str)
        return self.phrase_retrieve(query)

    def query_rank(self, query_str):
        """
        Given a string, process and then return the list of the top matching
        documents, rank-ordered.
        """
        query = self.process_query(query_str)
        return self.rank_retrieve(query)

## Running the code
You can use the `run_tests` and `run_query` functions to test your code. `run_tests` tests how different components your search engine code perform on a small set of queries and checks whether or not it matches up with our solution's results. `run_query` can be used to test your code on individual queries. 

Note that the first run for either of these functions might take a little while, since it will stem all the words in every document create a directory named stemmed/ in ../data/ProjectGutenberg/. This is meant to be a simple cache for the stemmed versions of the text documents. Later runs will be much faster after the first run since all the stemming will already be completed. However, this means that **if something happens during this first run and it does not get through processing all the documents, you may be left with an incomplete set of documents in ../data/ProjectGutenberg/stemmed/. If this happens, simply remove the stemmed/ directory and re-run!**

In [39]:
def run_tests(irsys):
    print("===== Running tests =====")

    ff = open('./data/dev_queries.txt')
    questions = [xx.strip() for xx in ff.readlines()]
    ff.close()
    ff = open('./data/dev_solutions.txt')
    solutions = [xx.strip() for xx in ff.readlines()]
    ff.close()

    epsilon = 1e-4
    for part in range(6):
        points = 0
        num_correct = 0
        num_total = 0

        prob = questions[part]
        soln = json.loads(solutions[part])
        

        if part == 0:  # inverted index test
            print("Inverted Index Test")
            queries = prob.split("; ")
            queries = [xx.split(", ") for xx in queries]
            queries = [(xx[0], int(xx[1])) for xx in queries]
            for i, (word, doc) in enumerate(queries):
                num_total += 1
                guess = irsys.get_word_positions(word, doc)
                if sorted(guess) == soln[i]:
                    num_correct += 1

        if part == 1:  # get postings test
            print("Get Postings Test")
            words = prob.split(", ")
            for i, word in enumerate(words):
                num_total += 1
                posting = irsys.get_posting_unstemmed(word)
                if posting == soln[i]:
                    num_correct += 1

        elif part == 2:  # boolean retrieval test
            print("Boolean Retrieval Test")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                guess = irsys.query_retrieve(query)
                if set(guess) == set(soln[i]):
                    num_correct += 1

        elif part == 3:  # phrase query test
            print("Phrase Query Retrieval")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                guess = irsys.phrase_query_retrieve(query)
                if set(guess) == set(soln[i]):
                    num_correct += 1

        elif part == 4:  # tfidf test
            print("TF-IDF Test")
            queries = prob.split("; ")
            queries = [xx.split(", ") for xx in queries]
            queries = [(xx[0], int(xx[1])) for xx in queries]
            for i, (word, doc) in enumerate(queries):
                num_total += 1
                guess = irsys.get_tfidf_unstemmed(word, doc)
                if guess >= float(soln[i]) - epsilon and \
                        guess <= float(soln[i]) + epsilon:
                    num_correct += 1

        elif part == 5:  # cosine similarity test
            print("Cosine Similarity Test")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                ranked = irsys.query_rank(query)
                top_rank = ranked[0]
                if top_rank[0] == soln[i][0]:
                    if top_rank[1] >= float(soln[i][1]) - epsilon and \
                            top_rank[1] <= float(soln[i][1]) + epsilon:
                        num_correct += 1

        feedback = "%d/%d Correct. Accuracy: %f" % \
                   (num_correct, num_total, float(num_correct) / num_total)

        if part == 1:
            if num_correct == num_total:
                points = 2
            elif num_correct >= 0.5 * num_total:
                points = 1
            else:
                points = 0
        elif part == 2:
            if num_correct == num_total:
                points = 1
            else:
                points = 0
        else:
            if num_correct == num_total:
                points = 3
            elif num_correct > 0.75 * num_total:
                points = 2
            elif num_correct > 0:
                points = 1
            else:
                points = 0

        print("    Score: %d Feedback: %s" % (points, feedback))

In [54]:
## Run this cell to run the tests
irsys = IRSystem()
irsys.read_data('./data/ProjectGutenberg')
irsys.index()
irsys.compute_tfidf()

run_tests(irsys)

Reading in documents...
Already stemmed!
Indexing...
Calculating tf-idf...
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 6/6 Correct. Accuracy: 1.000000
Get Postings Test
    Score: 2 Feedback: 6/6 Correct. Accuracy: 1.000000
Boolean Retrieval Test
    Score: 1 Feedback: 6/6 Correct. Accuracy: 1.000000
Phrase Query Retrieval
    Score: 3 Feedback: 6/6 Correct. Accuracy: 1.000000
TF-IDF Test
    Score: 3 Feedback: 6/6 Correct. Accuracy: 1.000000
Cosine Similarity Test
    Score: 3 Feedback: 6/6 Correct. Accuracy: 1.000000


In [55]:
def run_query(query):
    irsys = IRSystem()
    irsys.read_data('./data/ProjectGutenberg')
    irsys.index()
    irsys.compute_tfidf()
    
    print("Best matching documents to '%s':" % query)
    results = irsys.query_rank(query)
    for docId, score in results:
        print("%s: %e" % (irsys.titles[docId], score))
        
# Run any query you want!
run_query("My very own query")

Reading in documents...
Already stemmed!
Indexing...
Calculating tf-idf...
Best matching documents to 'My very own query':
Anna Karenina - Leo Tolstoy: 1.382642e-02
The Scarlet Letter - Nathaniel Hawthorne: 1.199004e-02
Around the World in Eighty Days - Jules Verne: 1.155807e-02
Dracula - Bram Stoker: 1.010096e-02
Violin Mastery_Talks with Master Violinists and Teachers - Frederick Herman Martens: 9.957106e-03
The Iliad - Homer: 8.318973e-03
War and Peace - Leo Tolstoy: 5.910470e-03
Don Quixote - Miguel de Cervantes Saavedra: 5.333663e-03
Our Vanishing Wild Life_Its Extermination and Preservation - William T Hornaday: 5.221044e-03
Les Misérables - Victor Hugo: 5.050607e-03
The Count of Monte Cristo - Alexandre Dumas and Auguste Maquet: 4.987717e-03
Anthem - Ayn Rand: 1.705823e-04
The Sorrows of Young Werther - Johann Wolfgang von Goethe: 1.108843e-04
The Fables of Aesop - Aesop: 1.040306e-04
Frankenstein or the Modern Prometheus - Mary Wollstonecraft Shelley: 1.037547e-04
The Time Mach

For reference, you can run the cell below to see the titles of all the documents in our dataset. As sanity checks, you can try tailoring your queries in `run_query` to output certain titles and/or checking what IR system outputs against the list of titles to see if the results make sense (i.e. the book _Great Pianists on Piano Playing_ should probably be among the top results if the query is "pianists play piano").

In [56]:
# Prints full list of book titles in dataset (in alphabetical order)
for i in range(len(irsys.titles)):
    print("%d. %s" % (i + 1, irsys.titles[i]))

1. A Handbook of Laboratory Glass-Blowing - Bernard D Bolas
2. A Journey to the Centre of the Earth - Jules Verne
3. Anna Karenina - Leo Tolstoy
4. Antarctic Penguins-A Study of Their Social Habits - G Murray Levick
5. Anthem - Ayn Rand
6. Around the World in Eighty Days - Jules Verne
7. Autobiography - John Stuart Mill
8. Butterflies Worth Knowing - Clarence Moores Weed
9. Concerning the Spiritual in Art - Wassily Kandinsky
10. Crime and Punishment - Fyodor Dostoyevsky
11. David Copperfield - Charles Dickens 
12. Divine Comedy - Dante Alighieri
13. Don Quixote - Miguel de Cervantes Saavedra
14. Dracula - Bram Stoker
15. Encyclopedia of Needlework - Thérèse de Dillmont
16. Frankenstein or the Modern Prometheus - Mary Wollstonecraft Shelley
17. Getting Acquainted with the Trees - J Horace McFarland
18. Great Pianists on Piano Playing - James Francis Cooke
19. Handbook of Wool Knitting and Crochet - Anonymous
20. Hippolytus and the Bacchae - Euripides
21. Les Misérables - Victor Hugo
22.

## Information Retrieval Systems and Search Engines

Safiya Umoja Noble’s <a href="https://nyupress.org/9781479837243/algorithms-of-oppression/">Algorithms of Oppression</a> (2018) provides insight into how search engines and information retrieval algorithms can exhibit substantial racist and sexist biases. Noble demonstrates how prejudice against black women is embedded into search engine ranked results. These biases are apparent in both Google search’s autosuggestions and the first page of Google results. In this assignment, we have explored numerous features that are built into information retrieval systems, like Google Search. 

How could the algorithms you built in this assignment contribute to enforcing real-world biases? 

How can we reduce data discrimination and algorithmic bias that perpetuate gender and racial inequalities in search results and IR system?

Please provide a short response to each of these questions (1-2 paragraphs per question).

In [57]:
class IRSystem(IRSystem):
    def algorithmic_bias_in_IR_systems(self):
        # TODO: Place your response into the response string below
        response = "One way in which the algorithms we built can enforce real-world biases is in our training data (documents). Specifically, if we have a set of documents that we compare queries to not be equally represntative of everything in the real world, then the same biases will persist. If we choose documents that have abnormally high counts of a specific word than another, then when querying, the tf-idf weights will be extremely unbalanced and biased. The cosine similarity based ranking metric that we built automatically defaults some values to 0 if we encounter an out of vocabulary term. This is not a good practice as it will severly underrepresent minority group terms, etc. One way to reduce data discrimination and algorithmic bias is by creating a more representative training dataset. Ensure that all the training documents are diverse and broad. As an extra layer, include something that weights the tf-idf terms or ranking similarities based on the distribution of our training data. "
        return response

Once you're ready to submit, you can run the cell below to prepare and zip
up your solution:

In [1]:
%%bash

if [[ ! -f "./pa4.ipynb" ]]
then
    echo "WARNING: Did not find notebook in Jupyter working directory. This probably means you're running on Google Colab. You'll need to go to File->Download .ipynb to download your notebok and other files, then zip them locally. See the README for more information."
else
    echo "Found notebook file, creating submission zip..."
    zip -r submission.zip pa4.ipynb deps/
fi

Found notebook file, creating submission zip...
	zip warning: name not matched: deps/
updating: pa4.ipynb (deflated 75%)


If you're running on Google Colab, see the README for instructions on
how to submit.

__Best of luck!__

__Some reminders for submission:__
* If you have any extra files required for your implementation to work, make
 sure they are in a `deps/` folder on the same level as `pa4.ipynb` and
 include that folder in your submission zip file.
 * Make sure you didn't accidentally change the name of your notebook file,
 (it should be `pa4.ipynb`) as that is required for the autograder to work.
* Go to Gradescope (gradescope.com), find the PA4 IR assignment and
upload your zip file (`submission.zip`) as your solution.
* Wait for the autograder to run (it should only take a minute or so) and check
that your submission was graded successfully! If the autograder fails, or you
get an unexpected score it may be a sign that your zip file was incorrect.